Librerías y Funciones propias:

In [1]:
import sys
import os
sys.path.append(os.path.abspath('..')) 

import numpy as np
import pyroomacoustics as pra
from scipy.io import wavfile
import pandas as pd
from algoritmos.CC import timeAveragedcrossCorrelation
from algoritmos.GCC import generalizedCrossCorrelation
from algoritmos.DOA import doa
from helpers.addNoise import addNoise
from helpers.downsampleSignal import downsampleSignal

Señales de entrada:

In [ ]:

fsOriginal, sourceSignalOriginal = wavfile.read("../audios/p262_001.wav")

if sourceSignalOriginal.dtype != np.float32 and sourceSignalOriginal.dtype != np.float64:
    sourceSignalOriginal = sourceSignalOriginal.astype(np.float32)

if np.max(np.abs(sourceSignalOriginal)) > 1:
    sourceSignalOriginal = sourceSignalOriginal / np.max(np.abs(sourceSignalOriginal))

sourceSignalsFs = []
fss = [16000, 22000, 41000, 44000]
sourceSignalsSnr = []
snrsDf = pd.read_excel("../datos-resultados/Datos.xlsx", engine='openpyxl', header=None)
snrs = snrsDf.iloc[12, 4:105].to_numpy()

for newFs in fss:
    downsampledSourceSignal, newFs = downsampleSignal(sourceSignalOriginal, fsOriginal, newFs)
    sourceSignalsFs.append(downsampledSourceSignal)

for snr in snrs:
    sourceSignalsSnr.append(addNoise(sourceSignalOriginal, snr))


Creamos los rooms:

In [3]:
roomsFs = []

for sourceSignal, fs in zip(sourceSignalsFs, fss):

    # Room dimensions (width * length)
    roomDimensions = [8, 8, 3]

    # Create a room
    rt60Target = 0.5
    eAbsorption, max_order = pra.inverse_sabine(rt60Target, roomDimensions)
    room = pra.ShoeBox(
        roomDimensions, fs=fs, materials=pra.Material(eAbsorption), max_order=max_order
    )

    # Sound source position (x,y,z)
    sourcePosition = [6.828, 5.328, 1.5]

    # Add the source to the room
    room.add_source(sourcePosition, signal=sourceSignal)

    # Define a linear Microphone Array: 4 mics in a row
    distanceBetweenMics = 0.15
    initialMicPosition = [3.775, 2.5, 1]

    # Calculate positions for 4 microphones
    micPositions = np.array([
        [initialMicPosition[0] + i * distanceBetweenMics, 
        initialMicPosition[1],
        initialMicPosition[2]]
        for i in range(4)
    ])

    # Add microphones to the room
    room.add_microphone_array(pra.MicrophoneArray(micPositions.T, room.fs))

    roomsFs.append(room)


roomsSnr = []

for sourceSignal in sourceSignalsSnr:

    # Room dimensions (width * length)
    roomDimensions = [8, 8, 3]

    # Create a room
    rt60Target = 0.5
    eAbsorption, max_order = pra.inverse_sabine(rt60Target, roomDimensions)
    room = pra.ShoeBox(
        roomDimensions, fs=48000, materials=pra.Material(eAbsorption), max_order=max_order
    )

    # Sound source position (x,y,z)
    sourcePosition = [6.828, 5.328, 1.5]

    # Add the source to the room
    room.add_source(sourcePosition, signal=sourceSignal)

    # Define a linear Microphone Array: 4 mics in a row
    distanceBetweenMics = 0.15
    initialMicPosition = [3.775, 2.5, 1]

    # Calculate positions for 4 microphones
    micPositions = np.array([
        [initialMicPosition[0] + i * distanceBetweenMics, 
        initialMicPosition[1],
        initialMicPosition[2]]
        for i in range(4)
    ])

    # Add microphones to the room
    room.add_microphone_array(pra.MicrophoneArray(micPositions.T, room.fs))

    roomsSnr.append(room)

Simulamos los rooms

In [4]:

micSignalsArraysFs = []

for room in roomsFs:
    # Run the room simulation
    room.simulate()

    # Get the microphone signals
    micSignals = room.mic_array.signals
    micSignalsArraysFs.append(micSignals)

micSignalsArraysSnr = []

for room in roomsSnr:
    # Run the room simulation
    room.simulate()

    # Get the microphone signals
    micSignals = room.mic_array.signals
    micSignalsArraysSnr.append(micSignals)

# Revisar un room simulado
room = roomsFs[0]

print("Sample rate:", room.fs)
print("Duración de la señal (s):", len(room.sources[0].signal) / room.fs)
print("Forma de micSignals:", micSignalsArraysFs[0].shape)

for i in range(4):
    mic = micSignalsArraysFs[0][i]
    print(f"Mic {i} - max: {np.max(np.abs(mic))}, min: {np.min(mic)}")

Sample rate: 16000
Duración de la señal (s): 2.3485625
Forma de micSignals: (4, 60566)
Mic 0 - max: 0.4683429002761841, min: -0.44318047165870667
Mic 1 - max: 0.5051679015159607, min: -0.5051679015159607
Mic 2 - max: 0.503486156463623, min: -0.503486156463623
Mic 3 - max: 0.4458872079849243, min: -0.4458872079849243


Algoritmos

Time Averaged Cross Correlation

In [ ]:
delays = []
for i in range(3):
    delays.append((timeAveragedcrossCorrelation(micSignals[i], micSignals[i + 1], returnDelay=True, graphs=False)[1]) / fs)

print("Delays between microphones (in seconds):", delays)

print("DOA time averaged cross correlation: ", doa(delays, distanceBetweenMics))

GCC: Classic

In [ ]:
distanceBetweenMics = [0.15, 0.30, 0.45, 0.15, 0.30, 0.15]

doaClassicFs = []

for micSignals, fs in zip(micSignalsArraysFs, fss):
    delays = []

    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='classic', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA classic cross correlation: ", doa(delays, distanceBetweenMics))

    doaClassicFs.append(doa(delays, distanceBetweenMics))

doaClassicSnr = []

for micSignals in micSignalsArraysSnr:
    delays = []
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='classic', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA classic cross correlation: ", doa(delays, distanceBetweenMics))

    doaClassicSnr.append(doa(delays, distanceBetweenMics))


dfFs = pd.DataFrame(doaClassicFs, columns=['DOA_Classic_Fs'])
dfSnr = pd.DataFrame(doaClassicSnr, columns=['DOA_Classic_Snr'])

with pd.ExcelWriter('../datos-resultados/doa_results.xlsx', mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    dfFs.to_excel(writer, sheet_name='DOA_Classic_Fs', index=False)
    dfSnr.to_excel(writer, sheet_name='DOA_Classic_Snr', index=False)

Delays between microphones (in seconds): [0.0001875, 0.000375, 0.000625, 0.0001875, 0.0004375, 0.00025]
DOA classic cross correlation:  61.750848826563065
Delays between microphones (in seconds): [0.0001818181818181818, 0.0004090909090909091, 0.0006818181818181819, 0.00022727272727272727, 0.00045454545454545455, 0.00022727272727272727]
DOA classic cross correlation:  60.383064746742484
Delays between microphones (in seconds): [0.0001951219512195122, 0.0003902439024390244, 0.0006585365853658537, 0.00021951219512195122, 0.00046341463414634144, 0.00024390243902439024]
DOA classic cross correlation:  60.14184868783884
Delays between microphones (in seconds): [0.0001818181818181818, 0.00038636363636363635, 0.0006590909090909091, 0.00020454545454545454, 0.00045454545454545455, 0.00022727272727272727]
DOA classic cross correlation:  61.4249941764007
Delays between microphones (in seconds): [0.00025, 0.0004318181818181818, 0.0007045454545454546, 0.00025, 0.0005, 0.00025]
DOA classic cross corr

GCC: Roth

In [ ]:
doaRothFs = []

for micSignals, fs in zip(micSignalsArraysFs, fss):
    delays = []
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='roth', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA Roth: ", doa(delays, distanceBetweenMics))

    doaRothFs.append(doa(delays, distanceBetweenMics))

doaRothSnr = []

for micSignals in micSignalsArraysSnr:
    delays = []
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='roth', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA Roth: ", doa(delays, distanceBetweenMics))

    doaRothSnr.append(doa(delays, distanceBetweenMics))


dfFs = pd.DataFrame(doaRothFs, columns=['DOA_Roth_Fs'])
dfSnr = pd.DataFrame(doaRothSnr, columns=['DOA_Roth_Snr'])

with pd.ExcelWriter('../datos-resultados/doa_results.xlsx', mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    dfFs.to_excel(writer, sheet_name='DOA_Roth_Fs', index=False)
    dfSnr.to_excel(writer, sheet_name='DOA_Roth_Snr', index=False)

Delays between microphones (in seconds): [0.0003125, 0.0005, 0.00075, 0.000125, -0.0006875, 0.00025]
DOA Roth:  70.83330077695585
Delays between microphones (in seconds): [0.0003181818181818182, 0.0005, 0.0009090909090909091, 0.00013636363636363637, -0.0007272727272727272, 0.00027272727272727274]
DOA Roth:  69.01519644124527
Delays between microphones (in seconds): [0.00031707317073170733, 0.0005365853658536586, 0.0009268292682926829, 0.00014634146341463414, 0.0006097560975609756, 0.0002926829268292683]
DOA Roth:  50.82970041520625
Delays between microphones (in seconds): [0.0003181818181818182, 0.0006136363636363636, 0.0009318181818181818, 0.00029545454545454547, 0.0006136363636363636, 0.00029545454545454547]
DOA Roth:  45.65816019640854
Delays between microphones (in seconds): [0.00034090909090909094, 0.0006818181818181819, 0.001, 0.0001590909090909091, 0.0006590909090909091, 0.0003181818181818182]
DOA Roth:  45.1642070165029
Delays between microphones (in seconds): [0.00034090909090

GCC: SCOT

In [ ]:
doaScotFs = []

for micSignals, fs in zip(micSignalsArraysFs, fss):
    delays = []
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='scot', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA SCOT: ", doa(delays, distanceBetweenMics))

    doaScotFs.append(doa(delays, distanceBetweenMics))

doaScotSnr = []

for micSignals in micSignalsArraysSnr:
    delays = []
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='scot', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA SCOT: ", doa(delays, distanceBetweenMics))

    doaScotSnr.append(doa(delays, distanceBetweenMics))


dfFs = pd.DataFrame(doaScotFs, columns=['DOA_SCOT_Fs'])
dfSnr = pd.DataFrame(doaScotSnr, columns=['DOA_SCOT_Snr'])

with pd.ExcelWriter('../datos-resultados/doa_results.xlsx', mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    dfFs.to_excel(writer, sheet_name='DOA_SCOT_Fs', index=False)
    dfSnr.to_excel(writer, sheet_name='DOA_SCOT_Snr', index=False)

Delays between microphones (in seconds): [0.0003125, 0.0005, 0.00075, 0.000125, -0.0006875, 0.00025]
DOA SCOT:  70.83330077695585
Delays between microphones (in seconds): [0.0003181818181818182, 0.0005, 0.0009090909090909091, 0.00013636363636363637, -0.0007272727272727272, 0.00027272727272727274]
DOA SCOT:  69.01519644124527
Delays between microphones (in seconds): [0.00031707317073170733, 0.0005365853658536586, 0.0009268292682926829, 0.00014634146341463414, 0.0006097560975609756, 0.0002926829268292683]
DOA SCOT:  50.82970041520625
Delays between microphones (in seconds): [0.0003181818181818182, 0.0006136363636363636, 0.0009318181818181818, 0.00029545454545454547, 0.0006136363636363636, 0.00029545454545454547]
DOA SCOT:  45.65816019640854
Delays between microphones (in seconds): [0.00034090909090909094, 0.0006818181818181819, 0.001, 0.00034090909090909094, 0.0006590909090909091, 0.0003181818181818182]
DOA SCOT:  40.183211891262914
Delays between microphones (in seconds): [0.00034090909

GCC: Phase

In [ ]:
doaPhaseFs = []

for micSignals, fs in zip(micSignalsArraysFs, fss):
    delays = []
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='phase', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA Phase: ", doa(delays, distanceBetweenMics))

    doaPhaseFs.append(doa(delays, distanceBetweenMics))

doaPhaseSnr = []

for micSignals in micSignalsArraysSnr:
    delays = []
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='phase', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA Phase: ", doa(delays, distanceBetweenMics))

    doaPhaseSnr.append(doa(delays, distanceBetweenMics))


dfFs = pd.DataFrame(doaPhaseFs, columns=['DOA_Phase_Fs'])
dfSnr = pd.DataFrame(doaPhaseSnr, columns=['DOA_Phase_Snr'])

with pd.ExcelWriter('../datos-resultados/doa_results.xlsx', mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    dfFs.to_excel(writer, sheet_name='DOA_Phase_Fs', index=False)
    dfSnr.to_excel(writer, sheet_name='DOA_Phase_Snr', index=False)

Delays between microphones (in seconds): [0.0003125, 0.0005, 0.00075, 0.00025, -0.0006875, 0.00025]
DOA Phase:  67.79032376127249
Delays between microphones (in seconds): [0.0003181818181818182, 0.0005, 0.0009090909090909091, 0.00013636363636363637, -0.0007272727272727272, 0.00027272727272727274]
DOA Phase:  69.01519644124527
Delays between microphones (in seconds): [0.00031707317073170733, 0.0025609756097560977, 0.0009268292682926829, 0.00014634146341463414, 0.0006097560975609756, 0.0002926829268292683]
Ignorado: delay=0.002561, distancia=0.300, argumento=2.928
DOA Phase:  50.564183170072724
Ignorado: delay=0.002561, distancia=0.300, argumento=2.928
Delays between microphones (in seconds): [0.0003181818181818182, 0.0006136363636363636, 0.0009318181818181818, 0.00029545454545454547, 0.0006136363636363636, 0.00029545454545454547]
DOA Phase:  45.65816019640854
Delays between microphones (in seconds): [0.00034090909090909094, 0.0006818181818181819, 0.001, 0.00034090909090909094, 0.0006590